In [ ]:
# %%
%load_ext autoreload
%autoreload 2

import torch
import pandas as pd
from einops import *

from language import Transformer

# Disable gradients and use half precision to speed up the analysis
torch.set_grad_enabled(False)
model = Transformer.from_pretrained(d_model=1024, n_layer=1, modifier="i5").cuda().half()
vocab = model.vocab

#### Computation of the diagonal of the path through the MLP and residual
Find the highest weights corresponding to preceding and following tokens of "girl"

*Recreation of Table 1 from the paper*

In [ ]:
diag = model.ube.diagonal(residual=True)

K = 10
token = "girl"
idx = vocab[token]

preceding = vocab[torch.topk(diag[idx], k=K).indices]
following = vocab[torch.topk(diag[:, idx], k=K).indices]

pd.DataFrame(dict(preceding=preceding, self=[token]*K, following=following))

#### We can also look at the top weights in general
*(Approximate) recreation of Table 6 from the paper*

In [ ]:
vocab.label_top_weights(diag.T, axes=["input", "output"], k=10)

#### Given an interaction matrix, we can also study the eigenvalues

In [ ]:
K = 10
idx = vocab["game"]

inter = model.ube.interaction(idx, residual=True).cpu().float()
vals, vecs = torch.linalg.eigh(inter)

df = pd.DataFrame()
for i in range(1, 6):
    df[f"comp-{i}"] = vocab[torch.topk(vecs[:, -i], k=K, largest=False).indices]
df